In [13]:
# 初始化环境变量
import os
from dotenv import load_dotenv, find_dotenv

# 读取本地/项目的环境变量。
# find_dotenv()寻找并定位.env文件的路径
# load_dotenv()读取该.env文件，并将其中的环境变量加载到当前的运行环境中
_ = load_dotenv(find_dotenv())


def get_openai_api_key() -> str:
    return os.environ.get("OPENAI_API_KEY")


def get_openai_api_base() -> str:
    return os.environ.get("OPENAI_API_BASE")


def get_openai_model_name() -> str:
    return os.environ.get("OPENAI_MODEL_NAME")


print(f"openai_api_key: {get_openai_api_key()}")
print(f"openai_api_base: {get_openai_api_base()}")
print(f"openai_model_name: {get_openai_model_name()}")

openai_api_key: sk-17b97551ccfe4f459ac09ece6d9b6175
openai_api_base: https://dashscope.aliyuncs.com/compatible-mode/v1
openai_model_name: qwen2.5-72b-instruct


In [14]:
# 创建一个Swarm对象
from openai import OpenAI
from swarm import Swarm, Agent

# 实例化客户端
swarm_client = Swarm(
    client=OpenAI(api_key=get_openai_api_key(), base_url=get_openai_api_base())
)


def instructions(context_variables: dict):
    name = context_variables.get("name", "User")
    return f"You are a helpful agent. Greet the user by name ({name})."


def print_account_details(context_variables: dict):
    user_id = context_variables.get("user_id", None)
    name = context_variables.get("name", None)
    print(f"Account Details: {name} {user_id}")
    return "Success"


account_agent = Agent(
    name="Account Agent",
    model=get_openai_model_name(),
    instructions=instructions,
    functions=[print_account_details],
)

context_variables = {"name": "James", "user_id": 123}

response = swarm_client.run(
    messages=[{"role": "user", "content": "Hi!"}],
    agent=account_agent,
    context_variables=context_variables,
    debug=True,
)

print(response.messages[-1]["content"])

response = swarm_client.run(
    messages=[{"role": "user", "content": "Print my account details!"}],
    agent=account_agent,
    context_variables=context_variables,
)
print(response.messages[-1]["content"])

[2024-12-23 14:33:01] Getting chat completion for...: [{'role': 'system', 'content': 'You are a helpful agent. Greet the user by name (James).'}, {'role': 'user', 'content': 'Hi!'}]
[2024-12-23 14:33:02] Received completion: ChatCompletionMessage(content='Hello James! How can I assist you today?', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)
[2024-12-23 14:33:02] Ending turn.
Hello James! How can I assist you today?
Account Details: James 123
Here are your account details, James:

Unfortunately, the specific details didn't come through in the function's response. If you could provide me with more information or if there's another function that can fetch detailed account information, I'd be happy to assist you further. Let me know how you would like to proceed!
